# Create paper list from .bib file.

## 1. Build bibtexparser

In [1]:
# coding = utf-8
import re
import bibtexparser as bp
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import convert_to_unicode

In [2]:
bibfile_name = "paperlist_10.bib"

In [3]:
with open(bibfile_name) as bibfile:
    parser = BibTexParser()
    parser.customization = convert_to_unicode
    bibdata = bp.load(bibfile, parser=parser)

## 2. Parse .bib file

In [4]:
# sort by year
biblist = bibdata.entries
# biblist.sort(key=lambda x: x['year'], reverse=True)

In [5]:
def bib_to_md(biblist, venue_kws=None, link_kws=None):
    bibmd = []
    for i in range(len(biblist)):
        # author
        author = biblist[i]['author'].replace(",", "")
        author = author.replace(" and", ",")
        # year
        year = biblist[i]['year']
        # title
        title = biblist[i]['title']
        # venue
        venue = ""
        for v_kw in venue_kws:
            if v_kw in biblist[i].keys():
                if "arXiv" in biblist[i][v_kw]:
                    venue = biblist[i][v_kw]
                    break
                else:
                    venue = re.sub(u"([^\u0020\u0041-\u005a\u0061-\u007a])", "", biblist[i][v_kw])
                    break
        
        bibmd_string = f"1. **{title}**. *{author}*. {venue} {year}."
        # link keywords
        for kw in link_kws:
            if kw in biblist[i]:
                bibmd_string += f" [{link_kws[kw]}]({biblist[i][kw]})"
        bibmd.append(bibmd_string)
        bibmd.append("\n")
    
    return bibmd

In [6]:
bibmd = bib_to_md(biblist, venue_kws={'booktitle', 'journal', 'venue'}, link_kws={"paper_link": "paper", "code_link": "code"})

## 3. Create .md file

In [7]:
from mdutils import Html
from mdutils.mdutils import MdUtils

In [8]:
file_name = "paperlist_10.md"
title = "Paper List of Adversarial Robustness in Graph Machine Learning"
mdFile = MdUtils(file_name, title)

In [9]:
mdFile.new_paragraph("This list collects papers related to the adversarial robustness in graph machine learning.")
mdFile.new_list(bibmd, marked_with=' ')
mdFile.create_md_file()

# 4. Create .md file with hierachical classes